## Image Augmentation

Cats v Dogs 로 다음처럼 모델링 하고, 학습시켜본다.

4 convolutional layers with 32, 64, 128 and 128 convolutions

train for 100 epochs

### 데이터 제너레이터를 통해 이미지를 증강한다.


```

train_datagen = ImageDataGenerator(
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
```

* rotation_range is a value in degrees (0–180), a range within which to randomly rotate pictures.
* width_shift and height_shift are ranges (as a fraction of total width or height) within which to randomly translate pictures vertically or horizontally.
* shear_range is for randomly applying shearing transformations.
* zoom_range is for randomly zooming inside pictures.
* horizontal_flip is for randomly flipping half of the images horizontally. This is relevant when there are no assumptions of horizontal assymmetry (e.g. real-world pictures).
* fill_mode is the strategy used for filling in newly created pixels, which can appear after a rotation or a width/height shift.



In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip
  


--2022-12-30 06:45:01--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.204.128, 172.217.203.128, 172.253.123.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   154MB/s    in 0.4s    

2022-12-30 06:45:01 (154 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [2]:
import zipfile

In [3]:
file = zipfile.ZipFile('/tmp/cats_and_dogs_filtered.zip')

In [4]:
file.extractall('/tmp')

In [5]:
base_dir = '/tmp/cats_and_dogs_filtered'

In [6]:
train_dir = '/tmp/cats_and_dogs_filtered/train'

In [7]:
test_dir = '/tmp/cats_and_dogs_filtered/validation'

In [8]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [9]:
def build_model():
  model = Sequential()
  model.add( Conv2D(16, (3,3) , activation='relu' , input_shape=(150,150,3)  ) )
  model.add( MaxPooling2D( (2,2) , 2 ) )
  model.add( Conv2D(32, (3,3), activation='relu'))
  model.add( MaxPooling2D( (2,2) , 2 ) )
  model.add( Conv2D(64, (3,3) , activation='relu'))
  model.add( MaxPooling2D( (2,2) , 2 ) )

  model.add(Flatten())
  model.add(Dense(512, 'relu'))
  model.add(Dense(1, 'sigmoid'))
  
  model.compile('rmsprop', 'binary_crossentropy', metrics=['accuracy'])
  return model

In [21]:
model = build_model()

In [22]:
# 이미지 데이터를 증강하여, 학습을 시키자.

In [23]:
from keras.preprocessing.image import ImageDataGenerator

In [24]:
train_datagen = ImageDataGenerator(rescale=1/255.0, rotation_range= 30, 
                                   width_shift_range= 0.4, 
                                   height_shift_range= 0.2, 
                                   shear_range= 0.3,
                                   zoom_range= 0.5, 
                                   horizontal_flip= True )

In [25]:
test_datagen = ImageDataGenerator(rescale=1/255.0 )

In [26]:
train_generator = train_datagen.flow_from_directory(train_dir, (150,150) ,
                                  class_mode='binary', 
                                  batch_size = 20)

Found 2000 images belonging to 2 classes.


In [27]:
test_generator = test_datagen.flow_from_directory(test_dir, (150, 150), 
                                        class_mode='binary',
                                        batch_size = 20)

Found 1000 images belonging to 2 classes.


In [28]:
epoch_history = model.fit(train_generator, epochs=15, 
                          validation_data = (test_generator) , 
                          steps_per_epoch= 100 )

Epoch 1/15
100/100 [==============================] - 79s 778ms/step - loss: 0.8242 - accuracy: 0.5385 - val_loss: 0.6816 - val_accuracy: 0.5170
Epoch 2/15
100/100 [==============================] - 78s 782ms/step - loss: 0.6889 - accuracy: 0.5645 - val_loss: 0.6394 - val_accuracy: 0.6070
Epoch 3/15
100/100 [==============================] - 76s 754ms/step - loss: 0.6764 - accuracy: 0.5805 - val_loss: 0.6563 - val_accuracy: 0.5500
Epoch 4/15
100/100 [==============================] - 76s 762ms/step - loss: 0.6678 - accuracy: 0.6010 - val_loss: 0.6309 - val_accuracy: 0.6370
Epoch 5/15
100/100 [==============================] - 80s 803ms/step - loss: 0.6580 - accuracy: 0.6400 - val_loss: 0.6227 - val_accuracy: 0.6530
Epoch 6/15
100/100 [==============================] - 77s 772ms/step - loss: 0.6372 - accuracy: 0.6255 - val_loss: 0.5985 - val_accuracy: 0.6610
Epoch 7/15
100/100 [==============================] - 77s 771ms/step - loss: 0.6530 - accuracy: 0.6085 - val_loss: 0.6377 - val_ac

### 차트 그려보자

In [18]:
import matplotlib.pyplot as plt